In [1]:
import heapq
import csv

In [ ]:
# nodes = [0, 1, 2, 3]
# edges = {0: [1, 3], 1: [2], 2: [0], 3: [2]}
# edgeInfo = {(0, 1): "G1", (0, 3): "G4", (1, 2): "G3", (2, 0): "G2", (3, 2): "G4", (0, 0): "local", (1, 1): "local", (2, 2): "local"}

In [179]:
nodes = []
edges = {}
edgeInfo = {}

In [180]:
def read_csv(path):
    with open(path, 'r') as file:
        csv_rd = csv.reader(file)
        for row in csv_rd:
            if row[0] == "from_city":
                continue
            from_city, from_id, to_city, to_id, route_id, enabled = row[0], row[1], row[3], row[4], row[12], row[14]
            from_tuple, to_tuple = (from_city, from_id), (to_city, to_id)
            if from_tuple == to_tuple:
                continue
            if from_tuple not in nodes:
                nodes.append(from_tuple)
                edgeInfo[(from_tuple, from_tuple)] = ["local", False]
            if to_tuple not in nodes:
                nodes.append(to_tuple)
                edgeInfo[(to_tuple, to_tuple)] = ["local", False]
            if from_tuple not in edges:
                edges[from_tuple] = []
            # if to_tuple not in edges:
            #     edges[to_tuple] = []
            if to_tuple not in edges[from_tuple]:
                edges[from_tuple].append(to_tuple)
            edgeInfo[(from_tuple, to_tuple)] = (route_id, enabled)

In [181]:
read_csv('CNRailway/processed_routes.csv')

In [121]:
edgeInfo

{(('beijing', '1001'), ('beijing', '1001')): ['local', False],
 (('langfang', '1007'), ('langfang', '1007')): ['local', False],
 (('beijing', '1001'), ('langfang', '1007')): ('G39', 'True'),
 (('tianjin', '1020'), ('tianjin', '1020')): ['local', False],
 (('beijing', '1001'), ('tianjin', '1020')): ('G217', 'True'),
 (('cangzhou', '1007'), ('cangzhou', '1007')): ['local', False],
 (('beijing', '1001'), ('cangzhou', '1007')): ('G35', 'True'),
 (('tangshan', '1007'), ('tangshan', '1007')): ['local', False],
 (('beijing', '1001'), ('tangshan', '1007')): ('G237', 'True'),
 (('dezhou', '1016'), ('dezhou', '1016')): ['local', False],
 (('beijing', '1001'), ('dezhou', '1016')): ('G19', 'True'),
 (('jinan', '1016'), ('jinan', '1016')): ['local', False],
 (('beijing', '1001'), ('jinan', '1016')): ('G11', 'True'),
 (('nanjing', '1012'), ('nanjing', '1012')): ['local', False],
 (('beijing', '1001'), ('nanjing', '1012')): ('G1', 'True'),
 (('zhenjiang', '1012'), ('zhenjiang', '1012')): ['local', Fa

In [194]:
start = ("shanghai", "1017")
end = ("beijing", "1001")

In [183]:
def dijkstra(start, end):
    visited = {}
    nodeInfo = {}
    for node in nodes:
        nodeInfo[node] = (float('inf'), "placeholder", node) # Transfer Count, Incoming Node, Self
    # nodeInfo = [(float('inf'), "placeholder")] * len(nodes) 
    nodeInfo[start] = (0, start, start)

    # for key in nodeInfo.keys():
    #     nodeInfo[key] = (nodeInfo[key][0], nodeInfo[key][1], key)
    
    candidate = [nodeInfo[start]]

    while len(candidate) > 0:
        currNode = heapq.heappop(candidate)
        if currNode[2] in visited:
            continue
        visited[currNode[2]] = 1

        if currNode[2] == end:
            break
        
        currIncoming = edgeInfo[(currNode[1], currNode[2])][0]
        for neighbor in edges[currNode[2]]:
            if not edgeInfo[(currNode[2], neighbor)][1]:
                continue
            if edgeInfo[(currNode[2], neighbor)][0] != currIncoming:
                if currNode[0] + 1 < nodeInfo[neighbor][0]:
                    nodeInfo[neighbor] = (currNode[0] + 1, currNode[2], neighbor)
            else:
                if currNode[0] < nodeInfo[neighbor][0]:
                    nodeInfo[neighbor] = (currNode[0], currNode[2], neighbor)
            heapq.heappush(candidate, nodeInfo[neighbor])
    
    if nodeInfo[end][0] > len(nodes):
        return [], -1

    result = []
    resultNode = end
    while resultNode != start:
        result.append(resultNode)
        resultNode = nodeInfo[resultNode][1]
    result.append(start)

    return list(reversed(result)), nodeInfo

In [ ]:
# dijkstra(start, end)

In [192]:
def get_routes(start, end, K):
    """
    Find K shortest paths using Yen's Algorithm.

    :param graph: Graph data structure representing nodes and edges.
    :param start: Starting node.
    :param end: Ending node.
    :param K: Number of shortest paths to find.
    :return: List of K shortest paths.
    """
    # Determine the shortest path from the start to the end.
    first_path, first_nodeInfo = dijkstra(start, end)

    if not first_path:
        return []

    A = [first_path]  # Store k-shortest paths
    An = [first_nodeInfo]

    for k in range(1, K):
        B = []
        Bn = []
        # Iterate over the previous path to find deviation points
        for i in range(0, len(A[k - 1]) - 1): # Where to spur from last node or first node!!!
            spur_node = A[k - 1][i]  # Current node as spur
            root_path = A[k - 1][:i + 1]  # From start to spur node

            # Temporarily remove edges and nodes from graph
            nodes_removed = set()
            for path in A:
                # if path[:i + 1] == root_path:
                if spur_node in path:
                    # inn = path.index(spur_node)
                    # for ind in range(inn, len(path) - 1):
                    #     edgeInfo[(path[ind], path[ind + 1])] = (edgeInfo[(path[ind], path[ind + 1])][0], False)
                    #     nodes_removed.add(path[ind])
                    ind = path.index(spur_node)
                    edgeInfo[(path[ind], path[ind + 1])] = (edgeInfo[(path[ind], path[ind + 1])][0], False)
                    nodes_removed.add(path[ind])

            # for node in root_path[:-1]:
            #     nodes_removed.add(node)
            #     nodes.get(node)[4] = False

            # Calculate spur path
            spur_path, nodeInfo = dijkstra(spur_node, end)

            # If spur path exists, combine with root path
            if len(spur_path) > 0:
                total_path = root_path + spur_path[1:]
                if total_path not in B:
                    B.append(total_path)
                    for ind in range(0, i + len(spur_path)):
                        if ind <= i:
                            nodeInfo[A[k - 1][ind]] = An[k - 1][A[k - 1][ind]]
                        else:
                            nodeInfo[spur_path[ind - i]] = (nodeInfo[spur_path[ind - i]][0] + An[k - 1][spur_path[ind - i]][0], nodeInfo[spur_path[ind - i]][1], nodeInfo[spur_path[ind - i]][2])
                    Bn.append(nodeInfo)

            # Restore edges
            for node in nodes_removed:
                for nextNode in edges[node]:
                    edgeInfo[(node, nextNode)] = (edgeInfo[(node, nextNode)][0], True)
        if not B:
            break

        # Sort by total cost and select the smallest one
        # print(Bn)
        # print(B)
        BBn = list(zip(Bn, B))
        BBn.sort(key = lambda x : x[0][end][0])  # Adjust sorting metric as needed
        currBn, currB = BBn.pop(0)
        A.append(currB)
        An.append(currBn)

    return A

In [195]:
get_routes(start, end, 5)

[[('shanghai', '1017'), ('nanjing', '1012'), ('beijing', '1001')],
 [('shanghai', '1017'),
  ('nanjing', '1012'),
  ('jinan', '1016'),
  ('beijing', '1001')],
 [('shanghai', '1017'),
  ('nanjing', '1012'),
  ('jinan', '1016'),
  ('cangzhou', '1007'),
  ('beijing', '1001')],
 [('shanghai', '1017'),
  ('nanjing', '1012'),
  ('jinan', '1016'),
  ('dezhou', '1016'),
  ('beijing', '1001')],
 [('shanghai', '1017'),
  ('nanjing', '1012'),
  ('jinan', '1016'),
  ('langfang', '1007'),
  ('beijing', '1001')]]